<a href="https://colab.research.google.com/github/kalyani234/AI_Agents_Projects/blob/CrewAI/CrewAI_Event_Management_Multi_Agent_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Project Description

 🤖 Event Management Multi-Agent Framework is an AI-powered system that uses three specialized agents—Venue, Logistics, and Marketing—to plan and manage events.

- The Venue Agent finds suitable venues
- The Logistics Agent confirms bookings and arranges essentials
- the Marketing Agent promotes the event.

Together, they automate event planning from venue selection to final outreach.

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guida

In [19]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [20]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("open_api")
os.environ["SERPER_API_KEY"] = userdata.get("serpi_api")

# required by CrewAI v0.28
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"


In [21]:
from crewai import Agent, Crew, Task

# CrewAI Tools

In [37]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Creating Agents

In [74]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [75]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event"
        "including catering and equipment"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [76]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

# Creating Venue Pydantic Object
- Create a class VenueDetails using Pydantic BaseModel.
- Agents will populate this object with information about different venues by creating different instances of it.



In [77]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
from typing import List

class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str
    event_schedule: str
    transport_options: List[str]

# Creating Tasks
- By using output_json, you can specify the structure of the output you want.
- By using output_file, you can get your output in a file.
- By setting human_input=True, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [78]:
venue_task = Task(
    description=(
        "Find a venue in {event_city} that meets the criteria for "
        "{event_topic}. The venue must support at least "
        "{expected_participants} participants.\n\n"
        "Include the event schedule: {event_schedule}.\n"
        "Also provide detailed transport options near the venue "
        "such as train stations, airports, metro access, and parking availability.\n\n"
        "You MUST populate all fields of the VenueDetails Pydantic model."
    ),
    expected_output=(
        "A complete VenueDetails Pydantic object including venue name, address, "
        "capacity, booking_status, event_schedule, and transport_options."
    ),
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    agent=venue_coordinator
)


In [79]:
logistics_task = Task(
    description=(
        "Review the selected venue details. The venue coordinator may mark "
        "booking_status as 'To be confirmed'. It is your responsibility to "
        "confirm venue booking availability for the date {tentative_date}.\n\n"
        "After confirming, update the booking status to 'Confirmed'.\n"
        "Then coordinate catering and equipment for {expected_participants} "
        "participants.\n"
        "Provide a detailed logistics confirmation summary including:\n"
        "- venue booking confirmation,\n"
        "- catering vendor,\n"
        "- equipment setup plan,\n"
        "- timing and delivery schedule.\n"
    ),
    expected_output=(
        "A complete logistics confirmation including venue booking status, "
        "catering arrangements, equipment setup, and scheduling details."
    ),
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)


In [80]:
marketing_task = Task(
    description=(
        "Promote the event {event_topic} based on the confirmed venue details "
        "and logistics plan. Ensure that you read the final venue information "
        "including:\n"
        "- venue name,\n"
        "- address,\n"
        "- event schedule,\n"
        "- transport options,\n"
        "- confirmed booking status.\n\n"
        "Create a promotional plan targeting at least {expected_participants} "
        "potential attendees. Include social media strategy, email outreach, "
        "and key messaging that highlights the venue and its accessibility."
    ),
    expected_output=(
        "A marketing report in markdown format including campaign strategy, "
        "expected reach, platform-specific messaging, and how the venue "
        "strengthens the event's appeal."
    ),
    async_execution=True,
    output_file="marketing_report.md",
    agent=marketing_communications_agent
)


# Creating the Crew
- Note: Since you set async_execution=True for logistics_task and marketing_task tasks
- Now the order for them does not matter in the tasks list.

In [81]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

Running the Crew
Set the inputs for the execution of the crew.






In [82]:
event_details = {
    'event_topic': "Global AI Leaders Summit 2025 in UK",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future Data and AI technologies.",
    'event_city': "London",
    'tentative_date': "2025-12-15",
    'expected_participants': 1500,
    'budget': 1200,
    'event_schedule' : '9am - 5pm',
    'venue_type': "Conference Hall"
}

In [83]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in London that meets the criteria for Global AI Leaders Summit 2025 in UK. The venue must support at least 1500 participants.

Include the event schedule: 9am - 5pm.
Also provide detailed transport options near the venue such as train stations, airports, metro access, and parking availability.

You MUST populate all fields of the VenueDetails Pydantic model.


> Entering new CrewAgentExecutor chain...
I need to find a suitable venue in London for the Global AI Leaders Summit 2025, which requires a capacity of at least 1500 participants. Additionally, I need to gather information about transport options near the venue, including train stations, airports, metro access, and parking availability. 

Action: Search the internet  
Action Input: {"search_query": "venues in London for 1500 participants Global AI Leaders Summit 2025"}  
 


Search results: Title: The AI Summit London
Link: https://london.theais

In [84]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

print(data)

{'name': 'Tobacco Dock', 'address': 'Tobacco Quay, Wapping Lane, London, E1W 2SF', 'capacity': 10000, 'booking_status': 'Available for booking', 'event_schedule': '09:00 - 17:00', 'transport_options': ['Shadwell Station (5 mins walk)', 'Wapping Station (6 mins walk)', 'DLR: Tower Hill Station (15 mins), Bank DLR Station (1 stop)', 'Overground: Canary Wharf DLR (15 mins), Shoreditch Overground (2 stops)', 'London City Airport (5.4 miles)', 'Heathrow Airport: Elizabeth Line to Whitechapel Station', 'Bus 100: Direct bus service to Tobacco Dock', 'Bus D3: Direct bus service to Tobacco Dock', 'Nearby parking options available, including Tower Bridge Parking and Minories Car Park.']}


# Display the final result as Markdown.

In [85]:
from IPython.display import Markdown
Markdown(result)

{
  "name": "Tobacco Dock",
  "address": "Tobacco Quay, Wapping Lane, London, E1W 2SF",
  "capacity": 10000,
  "booking_status": "Available for booking",
  "event_schedule": "09:00 - 17:00",
  "transport_options": [
    "Shadwell Station (5 mins walk)",
    "Wapping Station (6 mins walk)",
    "DLR: Tower Hill Station (15 mins), Bank DLR Station (1 stop)",
    "Overground: Canary Wharf DLR (15 mins), Shoreditch Overground (2 stops)",
    "London City Airport (5.4 miles)",
    "Heathrow Airport: Elizabeth Line to Whitechapel Station",
    "Bus 100: Direct bus service to Tobacco Dock",
    "Bus D3: Direct bus service to Tobacco Dock",
    "Nearby parking options available, including Tower Bridge Parking and Minories Car Park."
  ]
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')